In [1]:
import os
import sys
sys.path.append('/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/')
import argparse
import numpy as np
import nibabel as nib
import scipy.io as sio
import subprocess
from scipy.stats import zscore
from nibabel.nicom import dicomreaders
import pydicom as dicom  # type: ignore
import time
from glob import glob
import shutil
from nilearn.image import new_img_like
import joblib
import rtCommon.utils as utils
from rtCommon.utils import loadConfigFile
import pickle5 as pickle
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
# from rtCommon.fileClient import FileInterface
# import rtCommon.projectUtils as projUtils
# from rtCommon.imageHandling import readRetryDicomFromFileInterface, getDicomFileName, convertDicomImgToNifti


argParser = argparse.ArgumentParser()
argParser.add_argument('--config', '-c', default='sub002.ses5.toml', type=str, help='experiment file (.json or .toml)')
argParser.add_argument('--skipPre', '-s', default=0, type=int, help='skip preprocess or not')
argParser.add_argument('--skipGreedy', '-g', default=0, type=int, help='skip greedy or not')
argParser.add_argument('--testRun', '-t', default=None, type=int, help='testRun, can be [None,1,2,3,4,5,6,7,8]')
argParser.add_argument('--scan_asTemplate', '-a', default=1, type=int, help="which scan's middle dicom as Template?")

args = argParser.parse_args("")
from rtCommon.cfg_loading import mkdir,cfg_loading
# config="sub001.ses2.toml"
cfg = cfg_loading(args.config)
os.chdir("/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/")
cfg.session

/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  # Remove the CWD from sys.path while we load stuff.


conda env=/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud
toml filename=/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/projects/rtSynth_rt/conf/sub002.ses5.toml


5

In [11]:

def compareScore(cfg,testRun=None): 
    # 这个函数是从minimalClass修改过来的。目的是为了使用ses4的模型来对比ses5的前两个recognition run和后两个recognition run 的testing score
    # cfg 使用的是ses5的cfg，可以使用cfg.usingModel_dir 的模型，但是使用的是ses5的数据
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import sklearn
    import joblib
    import nibabel as nib
    import itertools
    from sklearn.linear_model import LogisticRegression

    def other(target):
        other_objs = [i for i in ['bed', 'bench', 'chair', 'table'] if i not in target]
        return other_objs
    def red_vox(n_vox, prop=0.1):
        return int(np.ceil(n_vox * prop))
    if 'milgram' in os.getcwd():
        main_dir='/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/'
    else:
        main_dir='/Volumes/GoogleDrive/My Drive/Turk_Browne_Lab/rtcloud_kp/'
    working_dir=main_dir
    os.chdir(working_dir)

    '''
    if you read runRecording for current session and found that there are only 4 runs in the current session, 
    you read the runRecording for previous session and fetch the last 4 recognition runs from previous session
    '''
    runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")

    actualRuns = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='recognition'))[0])]) # can be [1,2,3,4,5,6,7,8] or [1,2,4,5]
    print(f"actualRuns={actualRuns}")
    objects = ['bed', 'bench', 'chair', 'table']

    new_run_indexs=[]
    new_run_index=1 #使用新的run 的index，以便于后面的testRun selection的时候不会重复。正常的话 new_run_index 应该是1，2，3，4，5，6，7，8
    for ii,run in enumerate(actualRuns[2:]): # load behavior and brain data for current session
        t = np.load(f"{cfg.recognition_dir}brain_run{run}.npy")
        mask = np.load(f"{cfg.chosenMask}")
        t = t[:,mask==1]
        t = normalize(t)
        brain_data=t if ii==0 else np.concatenate((brain_data,t), axis=0)

        t = pd.read_csv(f"{cfg.recognition_dir}behav_run{run}.csv")
        t['run_num'] = new_run_index
        new_run_indexs.append(new_run_index)
        new_run_index+=1
        behav_data=t if ii==0 else pd.concat([behav_data,t])
    FEAT=brain_data
    print(f"FEAT.shape={FEAT.shape}")
    assert len(FEAT.shape)==2
    META=behav_data

    # convert item colume to label colume
    imcodeDict={
    'A': 'bed',
    'B': 'chair',
    'C': 'table',
    'D': 'bench'}
    label=[]
    for curr_trial in range(META.shape[0]):
        label.append(imcodeDict[META['Item'].iloc[curr_trial]])
    META['label']=label # merge the label column with the data dataframe

    # Which run to use as test data (leave as None to not have test data)
    # testRun = 0 # when testing: testRun = 2 ; META['run_num'].iloc[:5]=2
    def train4wayClf(META, FEAT):
        runList = np.unique(list(META['run_num']))
        print(f"runList={runList}")
        accList={}
        for testRun in runList:
            trainIX = META['run_num']!=int(testRun)
            testIX = META['run_num']==int(testRun)

            # pull training and test data
            trainX = FEAT[trainIX]
            testX = FEAT[testIX]
            trainY = META.iloc[np.asarray(trainIX)].label
            testY = META.iloc[np.asarray(testIX)].label

            # Train your classifier
            clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                        multi_class='multinomial').fit(trainX, trainY)
            
            # model_folder = cfg.trainingModel_dir
            # Save it for later use
            # joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))
            
            # Monitor progress by printing accuracy (only useful if you're running a test set)
            acc = clf.score(testX, testY)
            print("acc=", acc)
            accList[testRun] = acc
        print(f"new trained full rotation 4 way accuracy mean={np.mean(list(accList.values()))}")
        return accList
    # accList = train4wayClf(META, FEAT)
    
    # 获得full rotation的2way clf的accuracy 平均值 中文
    accs_rotation=[]
    print(f"new_run_indexs={new_run_indexs}")

    # 用所有数据训练要保存并且使用的模型：
    allpairs = itertools.combinations(objects,2)
    accs={}
    # Iterate over all the possible target pairs of objects
    for pair in allpairs:
        # Find the control (remaining) objects for this pair
        altpair = other(pair)
        for obj in pair:
            # foil = [i for i in pair if i != obj][0]
            for altobj in altpair:
                # establish a naming convention where it is $TARGET_$CLASSIFICATION
                # Target is the NF pair (e.g. bed/bench)
                # Classificationis is btw one of the targets, and a control (e.g. bed/chair, or bed/table, NOT bed/bench)
                naming = '{}{}_{}{}'.format(pair[0], pair[1], obj, altobj)

                trainIX = ((META['label']==obj) | (META['label']==altobj))
                testIX = ((META['label']==obj) | (META['label']==altobj))

                # pull training and test data
                trainX = FEAT[trainIX]
                testX = FEAT[testIX]
                trainY = META.iloc[np.asarray(trainIX)].label
                testY = META.iloc[np.asarray(testIX)].label

                assert len(np.unique(trainY))==2

                # Train your classifier
                # clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                            # multi_class='multinomial').fit(trainX, trainY)
                
                # Save it for later use
                clf = joblib.load(cfg.usingModel_dir +'/{}.joblib'.format(naming))
                
                # Monitor progress by printing accuracy (only useful if you're running a test set)
                acc = clf.score(testX, testY)
                print(naming, acc)
                accs[naming]=acc
    print(f"average 2 way clf accuracy={np.mean(list(accs.values()))}")

    return accs

from scipy.stats import zscore
def normalize(X):
    _X=X.copy()
    _X = zscore(_X, axis=0)
    _X[np.isnan(_X)]=0
    return _X
compareScore(cfg,testRun=None)

actualRuns=[1, 2, 15, 16]
FEAT.shape=(96, 1390)
new_run_indexs=[1, 2]
bedbench_bedchair 0.7083333333333334
bedbench_bedtable 0.5833333333333334
bedbench_benchchair 0.5833333333333334
bedbench_benchtable 0.8125
bedchair_bedbench 0.7083333333333334
bedchair_bedtable 0.5833333333333334
bedchair_chairbench 0.5833333333333334
bedchair_chairtable 0.8333333333333334
bedtable_bedbench 0.7083333333333334
bedtable_bedchair 0.7083333333333334
bedtable_tablebench 0.8125
bedtable_tablechair 0.8333333333333334
benchchair_benchbed 0.7083333333333334
benchchair_benchtable 0.8125
benchchair_chairbed 0.7083333333333334
benchchair_chairtable 0.8333333333333334
benchtable_benchbed 0.7083333333333334
benchtable_benchchair 0.5833333333333334
benchtable_tablebed 0.5833333333333334
benchtable_tablechair 0.8333333333333334
chairtable_chairbed 0.7083333333333334
chairtable_chairbench 0.5833333333333334
chairtable_tablebed 0.5833333333333334
chairtable_tablebench 0.8125
average 2 way clf accuracy=0.7048611111111

/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


{'bedbench_bedchair': 0.7083333333333334,
 'bedbench_bedtable': 0.5833333333333334,
 'bedbench_benchchair': 0.5833333333333334,
 'bedbench_benchtable': 0.8125,
 'bedchair_bedbench': 0.7083333333333334,
 'bedchair_bedtable': 0.5833333333333334,
 'bedchair_chairbench': 0.5833333333333334,
 'bedchair_chairtable': 0.8333333333333334,
 'bedtable_bedbench': 0.7083333333333334,
 'bedtable_bedchair': 0.7083333333333334,
 'bedtable_tablebench': 0.8125,
 'bedtable_tablechair': 0.8333333333333334,
 'benchchair_benchbed': 0.7083333333333334,
 'benchchair_benchtable': 0.8125,
 'benchchair_chairbed': 0.7083333333333334,
 'benchchair_chairtable': 0.8333333333333334,
 'benchtable_benchbed': 0.7083333333333334,
 'benchtable_benchchair': 0.5833333333333334,
 'benchtable_tablebed': 0.5833333333333334,
 'benchtable_tablechair': 0.8333333333333334,
 'chairtable_chairbed': 0.7083333333333334,
 'chairtable_chairbench': 0.5833333333333334,
 'chairtable_tablebed': 0.5833333333333334,
 'chairtable_tablebench': 

In [9]:
A={'bedbench_bedchair': 0.625, #average 2 way clf accuracy=0.6423611111111112
 'bedbench_bedtable': 0.5416666666666666,
 'bedbench_benchchair': 0.4583333333333333,
 'bedbench_benchtable': 0.7708333333333334,
 'bedchair_bedbench': 0.7083333333333334,
 'bedchair_bedtable': 0.5416666666666666,
 'bedchair_chairbench': 0.4583333333333333,
 'bedchair_chairtable': 0.75,
 'bedtable_bedbench': 0.7083333333333334,
 'bedtable_bedchair': 0.625,
 'bedtable_tablebench': 0.7708333333333334,
 'bedtable_tablechair': 0.75,
 'benchchair_benchbed': 0.7083333333333334,
 'benchchair_benchtable': 0.7708333333333334,
 'benchchair_chairbed': 0.625,
 'benchchair_chairtable': 0.75,
 'benchtable_benchbed': 0.7083333333333334,
 'benchtable_benchchair': 0.4583333333333333,
 'benchtable_tablebed': 0.5416666666666666,
 'benchtable_tablechair': 0.75,
 'chairtable_chairbed': 0.625,
 'chairtable_chairbench': 0.4583333333333333,
 'chairtable_tablebed': 0.5416666666666666,
 'chairtable_tablebench': 0.7708333333333334}

B={'bedbench_bedchair': 0.7083333333333334, # average 2 way clf accuracy=0.7048611111111112
 'bedbench_bedtable': 0.5833333333333334,
 'bedbench_benchchair': 0.5833333333333334,
 'bedbench_benchtable': 0.8125,
 'bedchair_bedbench': 0.7083333333333334,
 'bedchair_bedtable': 0.5833333333333334,
 'bedchair_chairbench': 0.5833333333333334,
 'bedchair_chairtable': 0.8333333333333334,
 'bedtable_bedbench': 0.7083333333333334,
 'bedtable_bedchair': 0.7083333333333334,
 'bedtable_tablebench': 0.8125,
 'bedtable_tablechair': 0.8333333333333334,
 'benchchair_benchbed': 0.7083333333333334,
 'benchchair_benchtable': 0.8125,
 'benchchair_chairbed': 0.7083333333333334,
 'benchchair_chairtable': 0.8333333333333334,
 'benchtable_benchbed': 0.7083333333333334,
 'benchtable_benchchair': 0.5833333333333334,
 'benchtable_tablebed': 0.5833333333333334,
 'benchtable_tablechair': 0.8333333333333334,
 'chairtable_chairbed': 0.7083333333333334,
 'chairtable_chairbench': 0.5833333333333334,
 'chairtable_tablebed': 0.5833333333333334,
 'chairtable_tablebench': 0.8125}

[15, 16]